# Model for Nature Conservancy Fisheries Kaggle Competition

#### Dependencies

In [ ]:
import fish_data as fd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import os

#### Helper functions

In [ ]:
help(fd)

#### Generate a list of filenames

In [ ]:
fish_filenames = fd.generate_filenames_list()
print("There are {} filenames in the master set list".format(len(fish_filenames)))

In [ ]:
fish_label_arr = fd.make_labels(fish_filenames, 'train/', '/img')
fish_label_arr.shape
fish_label_arr[0:5,:]

In [ ]:
valid_size = 75
files_train, files_val, y_train, y_val = train_test_split(fish_filenames, fish_label_arr, test_size = valid_size)
print([x.shape for x in [y_train, y_val]])

#### Debugging Graph and session calls with input pipeline

## Graph and Session Runs

#### Graph parameters

In [ ]:
#Preprocessing
std_y = 300
std_x = 500

# General
num_channels = 3
num_labels = 8
batch_size = 25
stddev = 0.2

# convolution
kernel_sizes = [12, 3, 3, 3, 3, 3]
conv_depths = [64, 128, 256, 512, 256, 128]
conv_strides = [4, 1, 1, 1, 1, 1]

pool_strides = [2, 2, 2, 2]

final_depth = conv_depths[-1]

#dropout
kp = 0.75

# fully connected
fc1_depth = 256
fc2_depth = 64

#regularization
beta = 1e-1 

# Learning rate
init_rate = 5e-3
#per_steps = 6000
#decay_rate = 0.75


#### Session parameters

In [ ]:
# epochs
num_epochs = 5
# path for tensorboard summary file to be written
logs_path = os.getcwd()+'/TB_logs'
#valid_every = 25

In [ ]:
%run -i 'GRAPH.py'

In [ ]:
%run -i 'SESSION.py'